# Introduction for Deep learning practicioners new to remote sensing

- explain that there are largely two types of datasets in torchgeo
- explain vision datasets, explain geodatasets (most of them do not contain raw satellite imagery, need to pair)
- highlight difference between them
- what is special about remote sensing imagery as compared to imagenet
- crs units explanation
- projection that transforms the surface of a 3D earth onto a 2D planar image that we are used to
- as mentioned, remote sensing datasets consist of large tiles, how to do tiling (process of splitting a larger tile into smaller patches or chips)
- explain how this works in torchgeo
- why the chips can be of non-constant size
- how to return constant size chips that can be part of a dataloader
- so how to construct a dataloader with geosampler, explain the arguments of the geoSampler
- how to create datasplits if they are not predefined or not available
- how to use torchgeo models with and without pytorch lightning modules

The aim of this tutorial is to make users with a Deep Learning background that are new to remote sensing familiar with the TorchGeo library. In general, remote sensing datasets often contain much higher dimensional images than usual image datasets. Since, deep learning models sometimes require a given image input dimension or face memory issues for large input images, a main objective of this tutorial will be to demonstrate how to do this for all types of TorchGeo Datasets.

It is recommended to run this notebook on Google Colab if you do not have your own GPU. Click the "Open in Colab" butto above to get started.

## Setup

Install TorchGeo.

In [3]:
#%pip install torchgeo

## Imports

Import libraries that we require for this tutorial. And set a temporary data directory from which to work.

In [24]:
import os
import tempfile
import torch

from torch.utils.data import DataLoader
from torchgeo.datasets import NAIP, ChesapeakeDE, stack_samples
from torchgeo.datasets.loveda import LoveDA
from torchgeo.datasets.utils import download_url
from torchgeo.samplers import RandomGeoSampler

data_root = tempfile.gettempdir()

ImportError: cannot import name 'stack_samples' from 'torchgeo.datasets' (/home/nils/.virtualenvs/torchGeoEnv/lib/python3.8/site-packages/torchgeo/datasets/__init__.py)

## TorchGeo Datasets

In TorchGeo, we differentiate between two types of datasets: Geospatial Datasets and Non-geospatial Datasets. While the former includes geospatial information that we will look at in more detail, the latter do not. List other differences... We will now take a closer look at Non-geospatial datasets before we turn our attention to Geospatial datasets.



## Non-geospatial Datasets

For people familiar with torchvision datasets and their combination with Pytorch DataLoaders, TorchGeo's Non-geospatial datasets will work the same way except that by design, TorchGeo returns a dictionary containing a sample of the chosen dataset. For example, to use the [LoveDA Land-Cover Dataset](https://arxiv.org/abs/2110.08733), a semantic segmentation dataset with RGB imagery, you can simply do the following:

In [15]:
loveda_root = os.path.join(data_root, "loveda")
ds = LoveDA(root=loveda_root, split="val", download=True)
dl = DataLoader(ds, batch_size=8)

val


2425958400it [05:39, 7144678.39it/s]                                 


Extracting /tmp/loveda/Val.zip to /tmp/loveda


In [19]:
for sample in dl:
    image = sample["image"]
    mask = sample["mask"]

print(image.shape)

torch.Size([5, 3, 256, 256])


where we use the smaller validation split because it downloads faster and it is just for demonstration purposes. 

### Transformation

 We can observe that the default image dimensions of this dataset are 1024x1024. If we would like to change this, we can pass a custom transformation - for example random cropping - when instantiating a dataset.

In [20]:
# custom image dataset size transform
class RandomCrop(object):
    """Crop the image and label to a given size"""

    def __init__(self, output_size):
        """Initialize a Random Crop Object for torchgeo samples

        Args:
            output_size (tuple or int): Desired output size. If int, square crop
            is made.
        """
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        """Random Crop the image and label

        Args:
            sample (dict): sample returned from torchgeo dataset
        """
        img, mask = sample["image"], sample["mask"].squeeze(0)
        h, w = img.shape[1:]
        new_h, new_w = self.output_size

        top = torch.randint(low=0, high=(h - new_h), size=(1,)).item()
        left = torch.randint(low=0, high=(w - new_w), size=(1,)).item()

        img = img[:, top : top + new_h, left : left + new_w]
        mask = mask[top : top + new_h, left : left + new_w]

        return {"image": img, "mask": mask}

Now, let's use this transform by passing it as an argument when instantiating the dataset. You can then see that the image and mask have the desired dimension of 256x256.

In [21]:
# use the transform and demonstrate
ds = LoveDA(root=loveda_root, split="val", transforms=RandomCrop(256))
dl = DataLoader(ds, batch_size=8)

# just return one sample here to demonstrate not ful iteration
for sample in dl:
    image = sample["image"]
    mask = sample["mask"]

print(image.shape)

val
torch.Size([5, 3, 256, 256])
torch.Size([5, 256, 256])


As, we can see, samples from the dataloader now have the desired dimensions. Hopefully, you now have a good understanding of how TorchGeo's Non-geospatial datasets work, how easy they are to use, and how you can adapt a dataset to your liking through custom transforms. We will now turn our attention to Geospatial datasets.

## Geospatial Datasets

Geospatial Datasets, implemented as GeoDataset class in TorchGeo, contain geospatial information such as latitude, longitude, coordinate stystem, and projection. Furthermore, Geospatial Datasets can contain Raster or Vector Data (Explain difference here). When working with remote sensory data such as satellite imagery, there occurs a projection that transforms the surface of the 3D Globe onto a 2D image that we are used to seeing. Additionally, the data should include the precise geographical location of a captured image. For this purpose cartographers have developed a framework called the coordinate reference system (CRS), which aims to display the spherical earh in just two dimensions. Due to this, all projections will yield some distortion of distance, area, or angular properties! A nice example of, how different CRS distort a given map, can be found here. There exist many different CRS (link here) that each have their advantages and disadvantages depending on the location, scale, and purpose of a map. As a consequence, TorchGeo's Geospatial datasets have a crs parameter, that lets you project the given dataset as you deem useful. 

Another substantial difference between TorchGeo's Geospatial Datasets and common Pytorch datasets is that the GeoDataset does not index the dataset with an integer, but rather with a Bounding Box consisting of *geographical* coordinates. More specifically, every GeoDataset has a dataset.index object which is an R-tree (link here) containing the Bounding Boxes and filenames of all raster or vector files in the dataset. In TorchGeo there exists a custom GeoSampler that takes care of the indexing into geospatial datsets.

Before we finally take a closer look at how to set up the data pipeline, it should be mentioned that a majority of TorchGeo's geospatial datasets do not contain both images and labels, but only one of the two. This means that if we are interested in a certain TorchGeo dataset that only contains labels and want to train a model to make predictions from raw satellite imagery, we need to pair this particular dataset with an available imagery dataset of the same resolution. In many cases, TorchGeo can do this automatically. 

### Datasets

Let us consider the data from [National Agriculture Imagery Program (NAIP)](https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/) and more specifically, just one state of it, Delaware. This dataset only contains labels, and hence, we need to pair it with satellite imagery from another source, for example [National Agriculture Imagery Program (NAIP)](https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/). Describe the process of choosing NAIP tiles here. 

In [23]:
naip_root = os.path.join(data_root, "naip")
naip_url = "https://naipblobs.blob.core.windows.net/naip/v002/de/2018/de_060cm_2018/38075/"
tiles = [
    "m_3807511_ne_18_060_20181104.tif",
    "m_3807511_se_18_060_20181104.tif",
    "m_3807512_nw_18_060_20180815.tif",
    "m_3807512_sw_18_060_20180815.tif",
]
for tile in tiles:
    download_url(naip_url + tile, naip_root)

naip = NAIP(naip_root)

581770240it [02:10, 4461486.57it/s]                               


590502912it [02:36, 3761604.56it/s]                               


574469120it [02:28, 3879767.84it/s]                               


569662464it [02:23, 3972849.49it/s]                               


Next, we can tell TorchGeo to automatically download corresponding Chesapeake labels, by passing in the crs and resolution argument from the NAIP dataset.

In [25]:
chesapeake_root = os.path.join(data_root, "chesapeake")

chesapeake = ChesapeakeDE(chesapeake_root, crs=naip.crs, res=naip.res, download=True)

287350784it [01:00, 4781939.91it/s]                               


Extracting /tmp/chesapeake/_DE_STATEWIDE.zip to /tmp/chesapeake


Another feature of TorchGeo are Union and IntersectionDataset, which allow us to combine two or more datasets into one, so that we can retrieve samples from it simultaneously. In this case we desire the intersection of our NAIP and Chesapeake dataset.

In [27]:
dataset = naip & chesapeake

TypeError: unsupported operand type(s) for &: 'NAIP' and 'ChesapeakeDE'

### Sampler

Unlike typical PyTorch Datasets, TorchGeo GeoDatsets are indexed using a bounding box consisting of latitude, longitude and time information. For this purpose TorchGeo provides a custom GeoSampler instead of the default sampler or batch_sampler from PyTorch.

In [ ]:
sampler = RandomGeoSampler(naip, size=1000, length=10)

The size argument for the GeoSampler is in crs units, *not* pixel units, and defines how large of an area the sampled bounding box that the dataset gets indexed with should be. The length parameter decides how many samples this GeoSampler should return in total. 

### DataLoader

In [ ]:
dataloader = DataLoader(dataset, sampler=sampler, collate_fn=stack_samples, batch_size=2)

Having the dataloader available, now we can proceed with our training pipeline in a familiar fashion. Where the dataloader will return batches of size 2 and a total of 10 samples as dictated by the length argument to the GeoSampler.

In [ ]:
for sample in dataloader:
    image = sample["image"]
    mask = sample["mask"]

Now, if we desire a specific image dimension for Geospatial datasets, we can adapt the size parameter in our GeoSampler. As previously mentioned, the size argument is in crs units and *not* pixel units. We can work around it by recognizing that , and implementing that accordingly. 

In [ ]:
desired_size_pix = 256
crs_size = desired_size_pix * dataset.res
sampler = RandomGeoSampler(naip, size=crs_size, length=10)

# just return one sample here for demonstration

dataloader = DataLoader(dataset, sampler=sampler, collate_fn=stack_samples)

for sample in dataloader:
    image = sample["image"]
    mask = sample["mask"]

You can now observe that the image and mask have the desired pixel resolution.

## Conclusion

Hopefully, you have a better understanding of how to work with both Non-geospatial and geospatial datasets from TorchGeo. This tutorial just offered a small insight into the many different features TorchGeo offers.